In [ ]:
# Print folder tree for giving pathimport os

def print_folder_tree(path, indent=""):
    for item in os.listdir(path):
        item_path = os.path.join(path, item)
        print(indent + "|-- " + item)
        if os.path.isdir(item_path):
            print_folder_tree(item_path, indent + "    ")

# Set your desired path
path = "/Volumes/backup/Study Data/LV103 - BersonHackett Business Study/st1 - Dance Studio"
print_folder_tree(path)


In [2]:
# Find all .hdf5 files in a folder
import os

def find_hdf5_files(folder_path):
    """
    Recursively searches for .hdf5 files in a given folder path and returns their paths.

    Parameters:
        folder_path (str): The path to the folder where the search will start.

    Returns:
        list of str: A list of full paths to files with the .hdf5 extension.
    """
    hdf5_files = []
    for root, dirs, files in os.walk(folder_path):
        for file in files:
            if file.endswith(".hdf5"):
                hdf5_files.append(os.path.join(root, file))
    return hdf5_files

# Using your specified path
folder_path = "/Volumes/backup/Study Data/LV103 - BersonHackett Business Study/st1 - Dance Studio"
hdf5_files = find_hdf5_files(folder_path)

# Print the found .hdf5 file paths
for path in hdf5_files:
    print(path)


In [3]:
import mne
import h5py
import numpy as np
import pandas as pd

In [5]:
# Load the data from the HDF5 file 

import h5py

def navigate_hdf5(file_path):
    with h5py.File(file_path, 'r') as file:
        print("Top-level groups and datasets:")
        print(list(file.keys()))
        print()
        navigate_group(file)

def navigate_group(group, prefix=""):
    for name, item in group.items():
        if isinstance(item, h5py.Dataset):
            print(prefix + '  |-', name, ':', item.shape, item.dtype)
        elif isinstance(item, h5py.Group):
            print(prefix + name + '/')
            navigate_group(item, prefix=prefix + '  |')

# Set the path to the HDF5 file
file_path = '/Volumes/backup/Study Data/LV103 - BersonHackett Business Study/st1 - Dance Studio/Sep10/081_group01/physiology/dancestudio_day00_group01_lv103_0812024.09.10_11.46.30.hdf5'

navigate_hdf5(file_path)
# print a long line
print('-' * 80)




Top-level groups and datasets:
['AsynchronData', 'RawData', 'SavedFeatues', 'Version']

AsynchronData/
  |  |- AsynchronSignalTypes : (1,) |S1824
  |  |- Time : (1, 1) int64
  |  |- TypeID : (1, 1) int32
  |  |- Value : (1, 1) int32
RawData/
  |  |- AcquisitionTaskDescription : (1,) |S5166
  |  |- DAQDeviceCapabilities : (1,) |S25871
  |  |- DAQDeviceDescription : (1,) |S357
  |  |- Samples : (656155, 6) float32
  |  |- SessionDescription : (1,) |S205
  |  |- SubjectDescription : (1,) |S282
SavedFeatues/
  |  |- NumberOfFeatures : (1,) int32
Version/
  |  |- Version : (1,) |S3
--------------------------------------------------------------------------------


In [8]:
# HDF5 file description
description = physiology_raw['RawData']['DAQDeviceDescription'][()]
print(description)

'''[b'<?xml version="1.0" encoding="utf-8" ?>\n
<DAQDeviceDescription xmlns:xsd="http://www.w3.org/2001/XMLSchema" xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance">\n
  <Unit>uV</Unit>\n
      <MinValue>-250000</MinValue>\n
          <MaxValue>250000</MaxValue>\n
              <Vendor>Guger Technologies</Vendor>\n
                  <Name>g.USBamp (UB-2012.10.22 UB-2012.10.23 )</Name>\n
                  </DAQDeviceDescription>']
'''

[b'<?xml version="1.0" encoding="utf-8" ?>\n<DAQDeviceDescription xmlns:xsd="http://www.w3.org/2001/XMLSchema" xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance">\n  <Unit>uV</Unit>\n  <MinValue>-250000</MinValue>\n  <MaxValue>250000</MaxValue>\n  <Vendor>Guger Technologies</Vendor>\n  <Name>g.USBamp (UB-2012.10.22 UB-2012.10.23 )</Name>\n</DAQDeviceDescription>']


In [6]:
file_path = '/Volumes/backup/Study Data/LV103 - BersonHackett Business Study/st1 - Dance Studio/Sep10/081_group01/physiology/dancestudio_day00_group01_lv103_0812024.09.10_11.46.30.hdf5'
physiology_raw = h5py.File(file_path)

samples = physiology_raw['RawData']['Samples']#[:]/ 10e6  # TODO slicing is for getting the data into a numpy array


In [8]:
%matplotlib qt
import matplotlib.pyplot as plt

# Plot the first row of samples
plt.plot(samples[1000:10000,0])
plt.title('First Row of Samples')
plt.xlabel('Sample Index')
plt.ylabel('Amplitude')
plt.show()

In [10]:
import neurokit2 as nk

In [12]:
# Process ecg
ecg_signals, info = nk.ecg_process(samples[1000:10000,0], sampling_rate=256)

nk.ecg_plot(ecg_signals[:3000], info)

In [13]:
# Extract ECG features
nk.ecg_intervalrelated(ecg_signals)

/Users/hanytawfik/miniforge3/envs/mne/lib/python3.11/site-packages/neurokit2/hrv/hrv_nonlinear.py:529: NeuroKitWarning: DFA_alpha2 related indices will not be calculated. The maximum duration of the windows provided for the long-term correlation is smaller than the minimum duration of windows. Refer to the `scale` argument in `nk.fractal_dfa()` for more information.
  warn(


,ECG_Rate_Mean,HRV_MeanNN,HRV_SDNN,HRV_SDANN1,HRV_SDNNI1,HRV_SDANN2,HRV_SDNNI2,HRV_SDANN5,HRV_SDNNI5,HRV_RMSSD,...,HRV_SampEn,HRV_ShanEn,HRV_FuzzyEn,HRV_MSEn,HRV_CMSEn,HRV_RCMSEn,HRV_CD,HRV_HFD,HRV_KFD,HRV_LZC
0,91.778481,[[167.37735849056602]],[[8.66741558298919]],[[nan]],[[nan]],[[nan]],[[nan]],[[nan]],[[nan]],[[7.828694753179583]],...,[[2.5649493574615367]],[[4.477643803497341]],[[1.439328722528385]],[[1.0435968174739092]],[[0.0]],[[0.0]],[[1.8915854127621117]],[[1.89902066156178]],[[2.952573814071125]],[[1.0807397084081507]]
